In [1]:
import sys, pickle, ast
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [2]:
problem_folder = '../problems/WSCP_time_dependent'
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Load data

In [3]:
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from utils.OptiHive import OptiHive
from utils.EM_algorithm import build_EM_df, EM_algorithm
from utils.filtering import maximal_component_selection

problem_folder = project_root / 'problems' / 'WSCP_time_dependent'

problem_path = Path().resolve() / problem_folder
sys.path.insert(0, str(problem_path))

candidates = list((problem_folder).glob('solvers_*.pkl'))
assert len(candidates) == 1, f"Expected 1 solvers_*.pkl, found {len(candidates)}: {candidates}"
solvers_file = candidates[0]
with open(solvers_file, 'rb') as file:
    solvers = pickle.load(file)

# Run boostrapping

In [4]:
optiHive = OptiHive(problem_folder)

In [ ]:
n_solvers = 50
n_instances = 50
n_validity_tests = 50
n_runs = 1000

results = {
    "optimal_rate": 0,
    "feasible_rate": 0,
    "executable_rate": 0,
    "interpretable_rate": 0,
}

for run_id in tqdm(range(n_runs)):
    ranking, results_em, optimal_sampled = optiHive.run_selection(n_solvers = n_solvers, n_instances = n_instances, n_validity_tests = n_validity_tests)
    selected_solver = solvers[ranking[0][0]]
    results["optimal_rate"] += selected_solver["correct"] / n_runs
    results["feasible_rate"] += selected_solver["feasible"] / n_runs
    results["executable_rate"] += selected_solver["executable"] / n_runs
    results["interpretable_rate"] += selected_solver["interpretable"] / n_runs

100%|██████████| 1000/1000 [01:33<00:00, 10.74it/s]


# Report

In [10]:
print('-'*15 + ' Backbone LLM ' + '-'*16)
print(f"Optimality rate: {np.mean([solver['correct'] for solver in solvers])}")
print(f"Feasibility rate: {np.mean([solver['feasible'] for solver in solvers])}")
print(f"Executability rate: {np.mean([solver['executable'] for solver in solvers])}")
print(f"Interpretability rate: {np.mean([solver['interpretable'] for solver in solvers])}")
print('-'*15 + ' OptiHive ' + '-'*20)
print(f"Optimality rate: {results['optimal_rate']:.3f}")
print(f"Feasibility rate: {results['feasible_rate']:.3f}")
print(f"Executability rate: {results['executable_rate']:.3f}")
print(f"Interpretability rate: {results['interpretable_rate']:.3f}")
print('-'*45)

--------------- Backbone LLM ----------------
Optimality rate: 0.03
Feasibility rate: 0.12
Executability rate: 0.95
Interpretability rate: 0.95
--------------- OptiHive --------------------
Optimality rate: 0.640
Feasibility rate: 0.749
Executability rate: 1.000
Interpretability rate: 1.000
---------------------------------------------
